In [3]:
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import numpy as np
from pathlib import Path

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

In [5]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [6]:
timeframe = "3Month"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-01-01", tz="America/New_York").isoformat()

# Set the stock tickers
tickers = ["SUSA", "XLY", "XLP", "FENY", "XME", "VIS", "XLV", "XLF", "VGT", "VOX", "VPU", "SCHH", "SPY", "QQQ"]

# Get 1 year's worth of historical data for all stocks
# HINT: Set "limit" to at least 10000 so all ticker rows are captured from get_bars()
df_ticker = api.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Display sample data
df_ticker.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2016-01-01 05:00:00+00:00,17.05,18.2200,14.140,17.56,11154476,48303,16.297547,FENY
2016-04-01 04:00:00+00:00,17.26,19.9699,17.010,19.40,12763160,41966,18.748095,FENY
2016-07-01 04:00:00+00:00,19.39,20.2052,18.200,19.83,10560153,37350,19.251545,FENY
2016-10-01 04:00:00+00:00,19.90,22.1200,18.715,21.18,13116922,43777,20.447934,FENY
2017-01-01 05:00:00+00:00,21.48,21.6299,18.900,19.63,16783541,63000,20.210885,FENY


In [7]:
df_ticker = df_ticker.reset_index()
df_ticker.head()

,timestamp,open,high,low,close,volume,trade_count,vwap,symbol
0,2016-01-01 05:00:00+00:00,17.05,18.2200,14.140,17.56,11154476,48303,16.297547,FENY
1,2016-04-01 04:00:00+00:00,17.26,19.9699,17.010,19.40,12763160,41966,18.748095,FENY
2,2016-07-01 04:00:00+00:00,19.39,20.2052,18.200,19.83,10560153,37350,19.251545,FENY
3,2016-10-01 04:00:00+00:00,19.90,22.1200,18.715,21.18,13116922,43777,20.447934,FENY
4,2017-01-01 05:00:00+00:00,21.48,21.6299,18.900,19.63,16783541,63000,20.210885,FENY


In [8]:

import numpy as np

#Create a column for the timestamp with datetime indexing 
df_ticker['timestamp'] = pd.to_datetime(df_ticker['timestamp'])
                                        
                                        
#Create a function that assigns a season based on the month index in the timestamp colum
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

#Create a new column labeled 'season' that implies date time as month and applies the get_season function
df_ticker['season'] = df_ticker['timestamp'].dt.month.apply(get_season)

#Display the df_ticker DataFrame
df_ticker.head()

,timestamp,open,high,low,close,volume,trade_count,vwap,symbol,season
0,2016-01-01 05:00:00+00:00,17.05,18.2200,14.140,17.56,11154476,48303,16.297547,FENY,Winter
1,2016-04-01 04:00:00+00:00,17.26,19.9699,17.010,19.40,12763160,41966,18.748095,FENY,Spring
2,2016-07-01 04:00:00+00:00,19.39,20.2052,18.200,19.83,10560153,37350,19.251545,FENY,Summer
3,2016-10-01 04:00:00+00:00,19.90,22.1200,18.715,21.18,13116922,43777,20.447934,FENY,Fall
4,2017-01-01 05:00:00+00:00,21.48,21.6299,18.900,19.63,16783541,63000,20.210885,FENY,Winter


In [9]:
#Set the index to timestamp column
df_ticker.set_index('timestamp', inplace=True)

#Create a new DataFrame by slicing df_ticker including the 'season', 'close', and 'symbol' column
season_df = df_ticker[['season', 'close', 'symbol']].copy()

#Display the new DataFrame
season_df.head()

,season,close,symbol
timestamp,,,
2016-01-01 05:00:00+00:00,Winter,17.56,FENY
2016-04-01 04:00:00+00:00,Spring,19.40,FENY
2016-07-01 04:00:00+00:00,Summer,19.83,FENY
2016-10-01 04:00:00+00:00,Fall,21.18,FENY
2017-01-01 05:00:00+00:00,Winter,19.63,FENY


In [10]:
# Reorganize the DataFrame
# Separate ticker data
SUSA = season_df[season_df['symbol']=='SUSA'].drop('symbol', axis=1)
XLY = season_df[season_df['symbol']=='XLY'].drop('symbol', axis=1)
XLP = season_df[season_df['symbol']=='XLP'].drop('symbol', axis=1)
FENY = season_df[season_df['symbol']=='FENY'].drop('symbol', axis=1)
XME = season_df[season_df['symbol']=='XME'].drop('symbol', axis=1)
VIS = season_df[season_df['symbol']=='VIS'].drop('symbol', axis=1)
XLV = season_df[season_df['symbol']=='XLV'].drop('symbol', axis=1)
XLF = season_df[season_df['symbol']=='XLF'].drop('symbol', axis=1)
VGT = season_df[season_df['symbol']=='VGT'].drop('symbol', axis=1)
VOX = season_df[season_df['symbol']=='VOX'].drop('symbol', axis=1)
VPU = season_df[season_df['symbol']=='VPU'].drop('symbol', axis=1)
SCHH = season_df[season_df['symbol']=='SCHH'].drop('symbol', axis=1)
SPY = season_df[season_df['symbol']=='SPY'].drop('symbol', axis=1)
QQQ = season_df[season_df['symbol']=='QQQ'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
season_df = pd.concat([SUSA, XLY, XLP, FENY, XME, VIS, XLV, XLF, VGT, VOX, VPU, SCHH, SPY, QQQ], axis=1, keys=["SUSA", "XLY", "XLP", "FENY", "XME", "VIS", "XLV", "XLF", "VGT", "VOX", "VPU", "SCHH", "SPY", "QQQ"])

# Display sample data
season_df.head()

SUSA             XLY            XLP         \
                           season   close  season  close  season  close   
timestamp                                                                 
2016-01-01 05:00:00+00:00  Winter  85.120  Winter  79.10  Winter  53.06   
2016-04-01 04:00:00+00:00  Spring  86.465  Spring  78.06  Spring  55.15   
2016-07-01 04:00:00+00:00  Summer  90.960  Summer  80.04  Summer  53.21   
2016-10-01 04:00:00+00:00    Fall  92.010    Fall  81.40    Fall  51.71   
2017-01-01 05:00:00+00:00  Winter  98.250  Winter  87.95  Winter  54.58   

                             FENY            XME         ...     VOX          \
                           season  close  season  close  ...  season   close   
timestamp                                                ...                   
2016-01-01 05:00:00+00:00  Winter  17.56  Winter  20.46  ...  Winter   92.72   
2016-04-01 04:00:00+00:00  Spring  19.40  Spring  24.41  ...  Spring   98.40   
2016-07-01 04:00:00+00:00  Summer  19.83  Summer  26.52  ...  Summer   94.39   
2016-10-01 04:00:00+00:00    Fall  21.18    Fall  30.41  ...    Fall  100.15   
2017-01-01 05:00:00+00:00  Winter  19.63  Winter  30.42  ...  Winter   94.87   

                              VPU            SCHH            SPY           \
                           season   close  season  close  season    close   
timestamp                                                                   
2016-01-01 05:00:00+00:00  Winter  107.53  Winter  41.40  Winter  205.520   
2016-04-01 04:00:00+00:00  Spring  114.50  Spring  43.44  Spring  209.475   
2016-07-01 04:00:00+00:00  Summer  106.96  Summer  42.65  Summer  216.350   
2016-10-01 04:00:00+00:00    Fall  107.01    Fall  41.04    Fall  223.520   
2017-01-01 05:00:00+00:00  Winter  112.63  Winter  40.78  Winter  235.680   

                              QQQ          
                           season   close  
timestamp                                  
2016-01-01 05:00:00+00:00  Winter  109.20  
2016-04-01 04:00:00+00:00  Spring  107.54  
2016-07-01 04:00:00+00:00  Summer  118.72  
2016-10-01 04:00:00+00:00    Fall  118.48  
2017-01-01 05:00:00+00:00  Winter  132.38  

[5 rows x 28 columns]

In [63]:
df_closing_prices = pd.DataFrame()

# Fetch the closing prices for all the tickers
for ticker in tickers:
    df_closing_prices[ticker] = season_df[ticker]["close"]
    df_closing_prices['season'] = season_df[ticker]['season']

# Reorganize the columns in the df_closing_prices DataFrame
df_closing_prices = df_closing_prices[['season', 'SUSA', 'XLY', 'XLP', 'FENY', 'XME', 'VIS', 'XLV', 'XLF', 'VGT', 'VOX', 'VPU', 'SCHH', 'SPY', 'QQQ']]

# Display sample data
df_closing_prices.head()


# Sort the data by seasons
df_closing_prices = df_closing_prices.sort_values(by='season')

# Display the df_closing_prices DataFrame
df_closing_prices.head()


,season,SUSA,XLY,XLP,FENY,XME,VIS,XLV,XLF,VGT,VOX,VPU,SCHH,SPY,QQQ
timestamp,,,,,,,,,,,,,,,
2022-10-01 04:00:00+00:00,Fall,82.32,129.16,74.55,23.84,49.81,182.64,135.85,34.20,319.41,82.25,153.36,19.29,382.43,266.28
2021-10-01 04:00:00+00:00,Fall,106.25,204.44,77.11,15.18,44.78,202.96,140.89,39.05,458.17,135.56,156.40,52.69,474.96,397.85
2016-10-01 04:00:00+00:00,Fall,92.01,81.40,51.71,21.18,30.41,119.26,68.94,23.26,121.50,100.15,107.01,41.04,223.52,118.48
2020-10-01 04:00:00+00:00,Fall,82.34,160.77,67.44,10.15,33.44,169.99,113.44,29.48,353.79,120.20,137.22,37.97,374.25,313.74
2017-10-01 04:00:00+00:00,Fall,111.07,98.69,56.89,20.05,36.37,142.37,82.68,27.91,164.73,91.15,116.59,41.62,266.94,155.76


In [64]:
# Create a new DataFrame that groupsby the season and calculates the Daily Returns for each ticker
grouped_df= df_closing_prices.groupby('season')
df_closing_prices['SUSA Daily Returns'] = grouped_df['SUSA'].pct_change()
df_closing_prices['XLY Daily Returns'] = grouped_df['XLY'].pct_change()
df_closing_prices['XLP Daily Returns'] = grouped_df['XLP'].pct_change()
df_closing_prices['FENY Daily Returns'] = grouped_df['FENY'].pct_change()
df_closing_prices['XME Daily Returns'] = grouped_df['XME'].pct_change()
df_closing_prices['VIS Daily Returns'] = grouped_df['VIS'].pct_change()
df_closing_prices['XLV Daily Returns'] = grouped_df['XLV'].pct_change()
df_closing_prices['XLF Daily Returns'] = grouped_df['XLF'].pct_change()
df_closing_prices['VGT Daily Returns'] = grouped_df['VGT'].pct_change()
df_closing_prices['VOX Daily Returns'] = grouped_df['VOX'].pct_change()
df_closing_prices['VPU Daily Returns'] = grouped_df['VPU'].pct_change()
df_closing_prices['SCHH Daily Returns'] = grouped_df['SCHH'].pct_change()
df_closing_prices['SPY Daily Returns'] = grouped_df['SPY'].pct_change()
df_closing_prices['QQQ Daily Returns'] = grouped_df['QQQ'].pct_change()


In [65]:
df_closing_prices.head()

,season,SUSA,XLY,XLP,FENY,XME,VIS,XLV,XLF,VGT,...,XME Daily Returns,VIS Daily Returns,XLV Daily Returns,XLF Daily Returns,VGT Daily Returns,VOX Daily Returns,VPU Daily Returns,SCHH Daily Returns,SPY Daily Returns,QQQ Daily Returns
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-10-01 04:00:00+00:00,Fall,82.32,129.16,74.55,23.84,49.81,182.64,135.85,34.20,319.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-01 04:00:00+00:00,Fall,106.25,204.44,77.11,15.18,44.78,202.96,140.89,39.05,458.17,...,-0.100984,0.111257,0.037100,0.141813,0.434426,0.648146,0.019823,1.731467,0.241953,0.494104
2016-10-01 04:00:00+00:00,Fall,92.01,81.40,51.71,21.18,30.41,119.26,68.94,23.26,121.50,...,-0.320902,-0.412397,-0.510682,-0.404353,-0.734815,-0.261213,-0.315793,-0.221105,-0.529392,-0.702199
2020-10-01 04:00:00+00:00,Fall,82.34,160.77,67.44,10.15,33.44,169.99,113.44,29.48,353.79,...,0.099638,0.425373,0.645489,0.267412,1.911852,0.200200,0.282310,-0.074805,0.674347,1.648042
2017-10-01 04:00:00+00:00,Fall,111.07,98.69,56.89,20.05,36.37,142.37,82.68,27.91,164.73,...,0.087620,-0.162480,-0.271157,-0.053256,-0.534385,-0.241681,-0.150343,0.096129,-0.286733,-0.503538


In [66]:

# Drop the old ticker columns from the df_closing_prices DataFrame
df_closing_prices.drop(columns=['SUSA','XLY','XLP','FENY','XME','VIS','XLV','XLF','VGT','VOX','VPU','SCHH','SPY','QQQ'],inplace=True)

# Sort the timestamp values and drop na's from the df_closing_prices DataFrame
df_closing_prices = df_closing_prices.dropna()
df_closing_prices = df_closing_prices.sort_values('timestamp', ascending=True)

In [67]:
# Display the df_closing_prices DataFrame
df_closing_prices.head(50)

,season,SUSA Daily Returns,XLY Daily Returns,XLP Daily Returns,FENY Daily Returns,XME Daily Returns,VIS Daily Returns,XLV Daily Returns,XLF Daily Returns,VGT Daily Returns,VOX Daily Returns,VPU Daily Returns,SCHH Daily Returns,SPY Daily Returns,QQQ Daily Returns
timestamp,,,,,,,,,,,,,,,
2016-01-01 05:00:00+00:00,Winter,-0.280169,-0.305226,-0.054358,-0.028761,-0.310647,-0.247963,-0.261253,-0.124854,-0.454070,0.110153,-0.170293,-0.068197,-0.272367,-0.392185
2016-04-01 04:00:00+00:00,Spring,-0.233500,-0.285819,0.070250,-0.087059,-0.311813,-0.217510,-0.140949,-0.139495,-0.409712,0.158738,-0.012591,0.038986,-0.227998,-0.373493
2016-07-01 04:00:00+00:00,Summer,-0.052006,-0.553970,-0.227048,0.376128,-0.364638,-0.409740,-0.433543,-0.485745,-0.700042,-0.329474,-0.231278,-0.066331,-0.495852,-0.668343
2016-10-01 04:00:00+00:00,Fall,-0.134024,-0.601839,-0.329400,0.395257,-0.320902,-0.412397,-0.510682,-0.404353,-0.734815,-0.261213,-0.315793,-0.221105,-0.529392,-0.702199
2017-01-01 05:00:00+00:00,Winter,0.118766,-0.476706,-0.200995,0.481509,-0.237976,-0.346278,-0.363029,-0.303084,-0.621663,-0.270287,-0.198420,-0.007544,-0.405344,-0.585185
2017-04-01 04:00:00+00:00,Spring,0.182675,0.148347,-0.003989,-0.073196,0.227776,0.208337,0.105223,0.079414,0.315155,-0.066768,-0.002096,-0.051335,0.154267,0.279896
2017-07-01 04:00:00+00:00,Summer,-0.154599,-0.253687,-0.121133,0.203939,0.264440,-0.080852,-0.093199,-0.077115,-0.294874,0.064129,-0.187052,-0.128047,-0.153746,-0.229649
2017-10-01 04:00:00+00:00,Fall,0.348919,-0.386142,-0.156435,0.975369,0.087620,-0.162480,-0.271157,-0.053256,-0.534385,-0.241681,-0.150343,0.096129,-0.286733,-0.503538
2018-04-01 04:00:00+00:00,Spring,-0.074383,-0.083054,-0.112623,0.235824,0.250705,-0.068519,-0.097329,-0.037681,-0.139741,-0.020418,-0.127661,-0.059181,-0.074241,-0.080808


In [68]:
import hvplot.pandas

#Create a bar plot for the df_closing_prices DataFrame
df_closing_prices.hvplot.bar(
    rot=90,
    groupby='season',
    ylabel='Daily Returns',
    xlabel='Year',
    title='Daily Returns by Season',
    hover_line_color='yellow',
    width=1500
).opts(
    yformatter='%.0f'
)


:DynamicMap   [season]
   :Bars   [timestamp,Variable]   (value)

In [69]:
from pathlib import Path

#Find and sort the values in a variable
temp_path = Path('Resources/US_Annual_Surface_Temp_Change8years.csv')

#Read the csv files and store in a variable
temp_df = pd.read_csv(temp_path,index_col='Fiscal Year')

#Display the DataFrame
temp_df 

,Avg Surface Temperature Change
Fiscal Year,
10/1/2015,1.531
10/1/2016,2.224
10/1/2017,1.433
10/1/2018,1.276
10/1/2019,1.034
10/1/2020,1.324
10/1/2021,1.144
10/1/2022,1.217


In [ ]:
df_closing_prices.drop(columns=['season'], inplace=True)

In [70]:
#Drop the season column from the df_closing_prices DataFrame
df_closing_prices.drop(columns=['season'], inplace=True)

#Create a new DataFrame df_2010_returns that slices the first 7 years of data from df_closing_prices
df_2010_returns = df_closing_prices[0:8]

#Sort df_2010 returns by the index
df_2010_returns.sort_index()

,SUSA Daily Returns,XLY Daily Returns,XLP Daily Returns,FENY Daily Returns,XME Daily Returns,VIS Daily Returns,XLV Daily Returns,XLF Daily Returns,VGT Daily Returns,VOX Daily Returns,VPU Daily Returns,SCHH Daily Returns,SPY Daily Returns,QQQ Daily Returns
timestamp,,,,,,,,,,,,,,
2016-01-01 05:00:00+00:00,-0.280169,-0.305226,-0.054358,-0.028761,-0.310647,-0.247963,-0.261253,-0.124854,-0.454070,0.110153,-0.170293,-0.068197,-0.272367,-0.392185
2016-04-01 04:00:00+00:00,-0.233500,-0.285819,0.070250,-0.087059,-0.311813,-0.217510,-0.140949,-0.139495,-0.409712,0.158738,-0.012591,0.038986,-0.227998,-0.373493
2016-07-01 04:00:00+00:00,-0.052006,-0.553970,-0.227048,0.376128,-0.364638,-0.409740,-0.433543,-0.485745,-0.700042,-0.329474,-0.231278,-0.066331,-0.495852,-0.668343
2016-10-01 04:00:00+00:00,-0.134024,-0.601839,-0.329400,0.395257,-0.320902,-0.412397,-0.510682,-0.404353,-0.734815,-0.261213,-0.315793,-0.221105,-0.529392,-0.702199
2017-01-01 05:00:00+00:00,0.118766,-0.476706,-0.200995,0.481509,-0.237976,-0.346278,-0.363029,-0.303084,-0.621663,-0.270287,-0.198420,-0.007544,-0.405344,-0.585185
2017-04-01 04:00:00+00:00,0.182675,0.148347,-0.003989,-0.073196,0.227776,0.208337,0.105223,0.079414,0.315155,-0.066768,-0.002096,-0.051335,0.154267,0.279896
2017-07-01 04:00:00+00:00,-0.154599,-0.253687,-0.121133,0.203939,0.264440,-0.080852,-0.093199,-0.077115,-0.294874,0.064129,-0.187052,-0.128047,-0.153746,-0.229649
2017-10-01 04:00:00+00:00,0.348919,-0.386142,-0.156435,0.975369,0.087620,-0.162480,-0.271157,-0.053256,-0.534385,-0.241681,-0.150343,0.096129,-0.286733,-0.503538


In [71]:
#Reset the index for temp_df and df_2010_returns
temp_df = temp_df.reset_index()
df_2010_returns = df_2010_returns.reset_index()

#Concatenated the temp_df and df_2010_returns DataFrames
temp_returns_df = pd.concat([temp_df, df_2010_returns], axis=1)

#Drop the 'timestamp' column from the
temp_returns_df.drop('timestamp', axis=1, inplace=True)

#Sort the temp_returns_df by index
temp_returns_df.sort_index()

#Drop na values from the temp_returns_df
temp_returns_df.dropna()

,Fiscal Year,Avg Surface Temperature Change,SUSA Daily Returns,XLY Daily Returns,XLP Daily Returns,FENY Daily Returns,XME Daily Returns,VIS Daily Returns,XLV Daily Returns,XLF Daily Returns,VGT Daily Returns,VOX Daily Returns,VPU Daily Returns,SCHH Daily Returns,SPY Daily Returns,QQQ Daily Returns
0,10/1/2015,1.531,-0.280169,-0.305226,-0.054358,-0.028761,-0.310647,-0.247963,-0.261253,-0.124854,-0.454070,0.110153,-0.170293,-0.068197,-0.272367,-0.392185
1,10/1/2016,2.224,-0.233500,-0.285819,0.070250,-0.087059,-0.311813,-0.217510,-0.140949,-0.139495,-0.409712,0.158738,-0.012591,0.038986,-0.227998,-0.373493
2,10/1/2017,1.433,-0.052006,-0.553970,-0.227048,0.376128,-0.364638,-0.409740,-0.433543,-0.485745,-0.700042,-0.329474,-0.231278,-0.066331,-0.495852,-0.668343
3,10/1/2018,1.276,-0.134024,-0.601839,-0.329400,0.395257,-0.320902,-0.412397,-0.510682,-0.404353,-0.734815,-0.261213,-0.315793,-0.221105,-0.529392,-0.702199
4,10/1/2019,1.034,0.118766,-0.476706,-0.200995,0.481509,-0.237976,-0.346278,-0.363029,-0.303084,-0.621663,-0.270287,-0.198420,-0.007544,-0.405344,-0.585185
5,10/1/2020,1.324,0.182675,0.148347,-0.003989,-0.073196,0.227776,0.208337,0.105223,0.079414,0.315155,-0.066768,-0.002096,-0.051335,0.154267,0.279896
6,10/1/2021,1.144,-0.154599,-0.253687,-0.121133,0.203939,0.264440,-0.080852,-0.093199,-0.077115,-0.294874,0.064129,-0.187052,-0.128047,-0.153746,-0.229649
7,10/1/2022,1.217,0.348919,-0.386142,-0.156435,0.975369,0.087620,-0.162480,-0.271157,-0.053256,-0.534385,-0.241681,-0.150343,0.096129,-0.286733,-0.503538


In [ ]:
new_temp_returns = temp_returns_df.dropna()

In [75]:

#Create a bar plot for the temp_returns_df DataFrame
temp_returns_df.hvplot.bar(
    rot=90,
    groupby='Fiscal Year',
    ylabel='Daily Returns',
    xlabel='Year',
    title='Daily Returns & Average Surface Temp Change',
    hover_line_color='yellow',
    width=1500
).opts(
    yformatter='%.0f'
)

:DynamicMap   [Fiscal Year]
   :Bars   [index,Variable]   (value)